In [1]:
import sys, os, git, time

import MDAnalysis as mda
from MDAnalysis.analysis import distances

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo

import numpy as np

In [2]:
ti = time.time()

self = MinimizeEnergy(maximum_steps=10,
    number_atoms=[50],
    epsilon=[0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 2,
    cut_off = 9,
    dump = 2)
self.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

step epot maxF
0 7859.547 67259.647
2 4892.332 40467.851
4 2719.643 20119.796
6 1379.022 7815.718
8 606.467 2231.298
10 204.932 442.707
time = 0.22 s


In [4]:
ti = time.time()

self = MonteCarlo(maximum_steps=1000,
    displace_mc = 1,
    number_atoms=[50, 100],
    epsilon=[0.1, 0.5],
    sigma = [3, 3],
    atom_mass = [1, 1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 100,
    cut_off = 9,
    dump = 2)
self.run()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

time = 23.26 s


In [7]:
self.positions

AttributeError: 'MonteCarlo' object has no attribute 'positions'